## Parte I

#### Responda las siguientes preguntas en 200 palabras o menos (2 puntos cada una)

1.	¿Qué es un proceso embarrassingly parallel y uno inherentemente serial? Dé un ejemplo de cada uno (distintos a los vistos en clase)

El proceso de convertir un programa serial o algoritmo en un programa paralelo es denominado paralelilazación, y los programas que pueden ser paralelizado por una simple división del trabajo entre los procesos son conocidos como “embarrassingly parallel”. Un ejemplo de ello es la renderización de infografías, utilizada en la animación por computadora, para la cual cada fotograma o píxel se puede renderizar de forma independiente (renderizado paralelo ). Sin embargo, no es posible que todos los procesos o algoritmos sean paralelizables, a estos se les denomina procesos inherentemente serial, por ejemplo: el muestreo de Gibbs (algoritmo de la cadena de Markov Monte Carlo - MCMC), usado para obtener una secuencia de observaciones que se aproximan a partir de una distribución de probabilidad multivariante, cuando el muestreo directo es difícil y usado comúnmente en la inferencia bayesiana. Este algoritmo es secuencial por naturaleza, es decir, se muestrea una variable aleatoria condicionada a todas las demás variables aleatorias. Si bien, este problema se puede absolver realizando un muestreo por bloques, es ineficiente pues implica un cálculo mucho más extenso

2.	¿Qué cuellos de botella puede enfrentar al paralelizar un proceso? Relaciónelo con la ley de Amdahl/Gustafson 

Uno de los principales cuellos de botella de un proceso de paralelización, es que podemos encontrar procesos que solo pueden ser paralelizables parcialmente, es decir que un parte del proceso sea “inherentemente en serie”, lo cual impide que la paralelización sea completa. En este sentido, la ley de Amdhl señala que a menos que se paralelice prácticamente todo el programa en serie, el aumento de la velocidad posiblemente sea muy limitado, incluso si el número de núcleos es elevado. Entonces, esta velocidad no podrá ser mayor a 1/r, incluso si r es bastante pequeño, no se podrá obtener una velocidad superior a r. Sin embargo, este problema no debe preocuparnos demasiado, pues la ley de Amdhl no toma en cuenta el tamaño de problema, el cual al ser más grande puede llegar a reducir la fracción “inherentemente serial” del programa, esta ley es conocida como la ley de Gustafson.

3.	¿En qué se diferencia un CPU de un GPU? Dé un ejemplo de un proceso que convendría paralelizar en cada uno de estos tipos de unidad de procesamiento. 

La principal diferencia se encuentra en la cantidad de núcleos que poseen, tanto un CPU y un GPU, los cuales condicionan las tareas que pueden o no cumplir. En cuanto al CPU tiene unos cuantos núcleos optimizados para el procesamiento en serie secuencial, convirtiéndolo en un potente motor de ejecución, que centra su reducido número de núcleos en realizar tareas individuales rápidamente. Por su parte, un GPU cuenta con una arquitectura en paralelo más grande que consiste de mieles de núcleos más pequeños y eficaces, y que se diseñaron para resolver varias tareas al mismo tiempo. 

4.	Piense en una tarea serial que le han encargado paralelizar. Describa el diseño de la implementación en paralelo de dicha tarea siguiente el método de Foster y los cuatro elementos que lo componen.



## Parte II

### Pregunta 1

#### Escribir un código (“parte2_1.py”) que realice lo siguiente:

a.	Que un procesador genere un diccionario y lo envíe a otros tres procesadores.

b.	Que cada uno de los tres procesadores reciba el diccionario enviado, imprima su número de procesador y el diccionario. 

c.	En otro chunk responda: Si usted ejecutara el código 10 veces, ¿el orden de los resultados sería siempre igual? ¿Por qué?

### Pregunta 2

Después de haber instalado MPI procedemos a importarlo así como las otras funciones que nos puedans servir como numpy

In [6]:
#from mpi4py import MPI
#import numpy as np
#import random
#import pandas as pd
from numpy import genfromtxt

Para comprobar que MPI se instaló correctamente, usamos el ejemplo visto en clase.

In [7]:
%%writefile mpi_example1.py 
from mpi4py import MPI

comm = MPI.COMM_WORLD # all available processors to communicate 
rank = comm.Get_rank() # give ranks to processors, local variable for e/ processor
size = comm.Get_size() # each processor identifies the total processors 

print("Hello World from rank ", rank, " out of ", size, " processors ")

Overwriting mpi_example1.py


In [ ]:
! mpiexec -n 4 python mpi_example1.py

a.Que un procesador genere dos numpy array diferentes, cada uno de 1,000,000 observaciones. Llamar “num1” y “num2” a estos numpy

Con nuestro procesador de rango 0 generamos dos arrays de números aleatorios con 1 000 000 de observaciones cada uno que llamaremos "num1" y "num2"

In [36]:
random.seed (1102)

In [46]:
%%writefile numpys.py 
from mpi4py import MPI

import numpy as np
import random

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    random.seed (1102)
    num1 = np.random.random(1000000)
    num2 = np.random.random(1000000)
    print (num1,num2)


Overwriting numpys.py


In [82]:
random.seed (1102)
! mpiexec -n 1 python numpys.py

[0.82230028 0.40343584 0.22223975 ... 0.75828752 0.9963173  0.32587117] [0.21657995 0.40071168 0.8840185  ... 0.27124061 0.89478879 0.59045959]


b.	Enviar cada numpy a un procesador diferente. Que cada uno de los otros procesadores reciba su numpy

Procedemos a enviar "num1" al procesador de rango 1 y "num2" al procesador de rango 2

In [99]:
%%writefile numpys2.py 
from mpi4py import MPI

import numpy as np
import random

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    random.seed (1102)
    num1 = np.random.random(1000000)
    num2 = np.random.random(1000000)
    comm.Send(num1, dest=1)
    comm.Send(num2, dest=2)
elif rank == 1:
    num1 = np.empty(1000000)
    comm.Recv(num1, source=0)
    print (num1)
elif rank == 2:
    num2 = np.empty(1000000)
    comm.Recv(num2, source=0)
    print (num2)

Overwriting numpys2.py


In [100]:
! mpiexec -n 3 python numpys2.py

[0.88771867 0.27119885 0.03070912 ... 0.19719056 0.82015062 0.89252686]
[0.29752897 0.40220258 0.1328891  ... 0.5105735  0.35169825 0.44080369]


c.	Que otro procesador (que no haya recibido nada) reciba “num1” y “num2” y los imprima.

Procedemos a enviar "num1" desde nuestro procesador de rango 1 y "num2" desde nuestro procesador de rango 2 a nuestro procesador de rango 3. 

In [68]:
%%writefile parte2_2.py 
from mpi4py import MPI

import numpy as np
import random

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    random.seed (1102)
    num1 = np.random.random(1000000)
    num2 = np.random.random(1000000)
    comm.Send(num1, dest=1)
    comm.Send(num2, dest=2)
elif rank == 1:
    num1 = np.empty(1000000)
    comm.Recv(num1, source=0)
    print (num1)
elif rank == 2:
    num2 = np.empty(1000000)
    comm.Recv(num2, source=0)
    print (num2)

Writing parte2_2.py


In [69]:
! mpiexec -n 4 python parte2_2.py

[0.91294557 0.08002201 0.44396812 ... 0.14826613 0.87352387 0.22030702]
[0.36044107 0.58287401 0.28592556 ... 0.93334727 0.01238815 0.27647252]


d.	Ejecute el código creado y registre el tiempo que toma realizar este ejercicio.

Procedemos a ejecutar nuestro último código tomando el tiempo de ejecución. 

In [71]:
import time
start = time.time()
! mpiexec -n 4 python parte2_2.py
end = time.time()
print(end - start)

[0.91259605 0.70382155 0.0011705  ... 0.01362625 0.87246215 0.29451386]
[0.65523925 0.93751911 0.93899528 ... 0.7779723  0.90110366 0.86416309]
0.7058086395263672


e.	En otro chunk responda: ¿existe una manera de agilizar este proceso con las herramientas de MPI? Sea detallado en su respuesta y argumentos.

### Pregunta 3

Generamos un numpy array que almacene el archivo: “tarea2.csv” y calculamos el máximo para tenerlo como referencia para las siguientes preguntas

In [73]:
from numpy import genfromtxt
tarea2 = genfromtxt('tarea2.csv', delimiter=',')
print(tarea2.shape)

(1048575,)


In [79]:
tarea2.max()

0.99999982

Generar un numpy array que almacene el archivo: “tarea2.csv”
a.	Escribir un código que halle el valor máximo de “tarea2” usando un procesador. Imprimir el valor máximo. Registrar el tiempo de demora.

In [77]:
%%writefile p3a.py 
from mpi4py import MPI

import numpy as np
import random
from numpy import genfromtxt

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    tarea2 = genfromtxt('tarea2.csv', delimiter=',')
    maximo = tarea2.max()
    print(maximo)

Overwriting p3a.py


In [78]:
import time
start = time.time()
! mpiexec -n 1 python p3a.py
end = time.time()
print(end - start)

5.873629093170166
0.99999982


Constatamos que encontramos el mismo máximo con nuestro código MPI que con el numpy array inicial

b.	Escribir un código que realice las siguientes indicaciones. Dividir el numpy en dos partes iguales. Que dos procesadores distintos encuentren el máximo de cada parte. Que otro procesador junte los máximos hallados y encuentre el máximo global. Este resultado debe ser igual al de 3a. Registrar el tiempo de demora.

In [83]:
tarea2_chunk = np.array_split(tarea2, 2)

In [94]:
tarea2_chunk

[array([0.34887171, 0.2668857 , 0.1366463 , ..., 0.57128704, 0.81662822,
        0.24393123]),
 array([0.71707726, 0.31591403, 0.57266182, ..., 0.48673952, 0.09359856,
        0.93271565])]

In [114]:
tarea2_chunk.size

AttributeError: 'list' object has no attribute 'size'

In [108]:
tarea2_chunk[0]

array([0.34887171, 0.2668857 , 0.1366463 , ..., 0.57128704, 0.81662822,
       0.24393123])

In [109]:
tarea2_chunk[0].size

524288

In [110]:
tarea2_chunk[1]

array([0.71707726, 0.31591403, 0.57266182, ..., 0.48673952, 0.09359856,
       0.93271565])

In [111]:
tarea2_chunk[1].size

524287

In [ ]:
tarea2_chunk = []
for chunk in pd.read_csv(tarea2 , chunks=2):
    chunks.append(tarea2)

In [127]:
%%writefile p3btest.py 
from mpi4py import MPI

import numpy as np
import pandas as pd
import random
from numpy import genfromtxt


comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    tarea2 = genfromtxt('tarea2.csv', delimiter=',')
    tarea2_chunk = np.array_split(tarea2, 2)
    chunk1=tarea2_chunk[0]
    chunk2=tarea2_chunk[1]
    comm.Send(chunk1, dest=1)
    comm.Send(chunk2, dest=2)
elif rank == 1:
    chunk1 = np.empty(524288)
    comm.Recv(chunk1, source=0)
    max1=chunk1.max()
    print(max1)
elif rank == 2:
    chunk2 = np.empty(524287)
    comm.Recv(chunk2, source=0)
    max2=chunk2.max()
    print(max2)

Overwriting p3btest.py


In [128]:
import time
start = time.time()
! mpiexec -n 3 python p3btest.py
end = time.time()
print(end - start)

0.99999791
0.99999982
4.333028554916382


In [119]:
%%writefile p3b.py 
from mpi4py import MPI

import numpy as np
import pandas as pd
import random
from numpy import genfromtxt


comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    tarea2 = genfromtxt('tarea2.csv', delimiter=',')
    tarea2_chunk = np.array_split(tarea2, 2)
    chunk1=tarea2_chunk[0]
    chunk2=tarea2_chunk[1]
    comm.Send(chunk1, dest=1)
    comm.Send(chunk2, dest=2)
elif rank == 1:
    chunk1 = np.empty(524288)
    comm.Recv(chunk1, source=0)
    max1=chunk1.max()
    comm.Send(max1, dest=2)
elif rank == 2:
    chunk2 = np.empty(524287)
    comm.Recv(chunk2, source=0)
    max2=chunk2.max()
    comm.Send(max2, dest=2)
elif rank == 3:
    max3 = np.empty(2)
    comm.Recv(max1, source=1)
    comm.Recv(max2, source=2)
    maxfin=max3.max()
    print(maxfin)

Overwriting p3b.py


In [120]:
import time
start = time.time()
! mpiexec -n 4 python p3b.py
end = time.time()
print(end - start)

^C
231.87553572654724


In [ ]:
  comm.Reduce(max3,maxfin,MPI.MAX)
    max3=chunk2.max()

c.	Repetir 3b dividiendo el numpy original en tres partes. Registrar el tiempo de demora.

d.	Comparar los tiempos registrados en 3a, 3b y 3c. ¿Hay una reducción del tiempo? ¿La reducción del tiempo es lineal? ¿Por qué?